<div>
    <table>
        <tr>
            <td>
                <center>
                    <h1>Premise Introduction</h1>
                     <a href="https://www.psi.ch/en/ta/people/romain-sacchi">Romain Sacchi</a> (PSI)
                    <br><br>
                    Duration: 1 hour 15 minutes.
                </center>
            </td>
        </tr>
    </div>

<div class="alert alert-info">
Note: we will be using <a href="https://docs.brightway.dev/en/latest/content/installation/index.html">Brightway 2</a>, not <a href="https://docs.brightway.dev/en/legacy/">Brightway 2.5</a>, because we would like to visualize our results in <a href="https://github.com/LCA-ActivityBrowser/activity-browser">Activity Browser</a>, which is for now only compatible with Brightway 2. But if you do not require ``activity-browser``, you can run this notebook in an environment containing ``brightway25``.
</div>

In [ ]:
from premise import *
import bw2data

Link to Premise scenario dashboard: https://premisedash-6f5a0259c487.herokuapp.com/

Premise documentation: https://premise.readthedocs.io/en/latest/

Examples notebook: https://github.com/polca/premise/blob/master/examples/examples.ipynb

In [ ]:
bw2data.projects.set_current("premise")

In [ ]:
bw2io.import_ecoinvent_release(
    version="3.10", 
    system_model="cutoff", # other options are "consequential", "apos" and "EN15804"
    username="xxxx",
    password="xxxx",
    biosphere_name="biosphere" # optional, otherwise a name is chosen for you
)

In [ ]:
ndb = NewDatabase(
    scenarios=[
        {"model":"image", "pathway":"SSP2-RCP19", "year":2050},
        {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2050},
    ],
    source_db="ecoinvent-3.10-cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.10", # <-- version of ecoinvent. Can be "3.8", "3.9" or "3.10". Must be a string.
    key='xxxxxxxxxxxxxxxxxxxxxxxxx', # <-- decryption key
    # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
    keep_source_db_uncertainty=False, # False by default, set to True if you want to keep ecoinvent's uncertainty data
    keep_imports_uncertainty=False, # False by default, set to True if you want to keep the uncertainty data of the additional inventories
    use_absolute_efficiency=True, # False by default, set to True if you want to use the IAM's absolute efficiencies
)

In [ ]:
# select sector to integrate, or all sectors
ndb.update()

In [ ]:
ndb.write_db_to_brightway()

In [ ]:
ndb.write_superstructure_db_to_brightway()

## Incremental Databases

The class ``IncrementalDatabase`` also implementing sectoral changes in an incremental, to easily distinguish the contribution of each sector on the results.

In [ ]:
ndb = IncrementalDatabase(
    scenarios=[
        {"model":"image", "pathway":"SSP2-RCP26", "year":2040},
    ],
    source_db="ecoinvent-3.10-cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.10", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key="xxxxxxxxxxxxxxxxxx",
)

In [ ]:
sectors = {
    "electricity": "electricity",
    "steel": "steel",
    "others": [
        "cement",
        "cars",
        "fuels"
    ]
}

In [ ]:
ndb.update(sectors=sectors)

In [ ]:
ndb.write_increment_db_to_brightway(name="test increment", file_format="csv")

## Reports
### Scenario report
You can generate a spreadsheet report showing the main variables of the scenario you have selected to create your databases. The report is saved in your working directory. Note that this report is generated automatically when exporting a database.

In [ ]:
ndb.generate_scenario_report()

## Changes report
You can generate a spreadsheet report of the changes made to the original database. It gives an overview on:
* the datasets created
* the datasets modified
* some performance indicators
* scaling factors used to scale certain exchanges

A "Validation" tab also shows any datasets that contain values or efficiencies that may seem incorrect.

The report is saved in your working directory. It is generated automatically when you export a database.

In [ ]:
ndb.generate_change_report()